In [2]:
### pipenv install scikit-learn pandas plotly matplotlib pingouin nbformat ipykernel optuna ipywidgets gradio

# EDA
import pandas as pd
import numpy as np
import pingouin as pg
import plotly.express as px
import plotly.figure_factory as ff
import matplotlib.pyplot as plt 

# ML
from sklearn.model_selection import cross_validate, StratifiedKFold, cross_val_predict, cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

# Hyperparameter optimization
import optuna


In [3]:
# Carregar o dataset
df_segmento = pd.read_csv('./datasets/dataset_segmento_cliente.csv')

### EDA

In [4]:
# visualizar os dados 
df_segmento.head(10)

,atividade_economica,faturamento_mensal,numero_de_funcionarios,localizacao,idade,inovacao,segmento_de_cliente
0,Comércio,713109.95,12,Rio de Janeiro,6,1,Bronze
1,Comércio,790714.38,9,São Paulo,15,0,Bronze
2,Comércio,1197239.33,17,São Paulo,4,9,Silver
3,Indústria,449185.78,15,São Paulo,6,0,Starter
4,Agronegócio,1006373.16,15,São Paulo,15,8,Silver
5,Serviços,1629562.41,16,Rio de Janeiro,11,4,Silver
6,Serviços,771179.95,13,Vitória,0,1,Starter
7,Serviços,707837.61,16,São Paulo,10,6,Silver
8,Comércio,888983.66,17,Belo Horizonte,10,1,Bronze
9,Indústria,1098512.64,13,Rio de Janeiro,9,3,Bronze


In [5]:
# Estrutura do dataset
df_segmento.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   atividade_economica     500 non-null    object 
 1   faturamento_mensal      500 non-null    float64
 2   numero_de_funcionarios  500 non-null    int64  
 3   localizacao             500 non-null    object 
 4   idade                   500 non-null    int64  
 5   inovacao                500 non-null    int64  
 6   segmento_de_cliente     500 non-null    object 
dtypes: float64(1), int64(3), object(3)
memory usage: 27.5+ KB


In [6]:
# valores possíveis - Variaveis categoricas
df_segmento['atividade_economica'].unique()

array(['Comércio', 'Indústria', 'Agronegócio', 'Serviços'], dtype=object)

In [7]:
# valores possíveis - Variaveis categoricas
df_segmento['localizacao'].unique()

array(['Rio de Janeiro', 'São Paulo', 'Vitória', 'Belo Horizonte'],
      dtype=object)

In [8]:
# valores possíveis - Variaveis categoricas
df_segmento['segmento_de_cliente'].unique()

array(['Bronze', 'Silver', 'Starter', 'Gold'], dtype=object)

In [9]:
# valores possíveis - Variaveis categoricas
df_segmento['inovacao'].unique()

array([1, 0, 9, 8, 4, 6, 3, 7, 5, 2])

In [10]:
# Distribuição da variavel segmento_de_cliente (target)
contagem_target = df_segmento['segmento_de_cliente'].value_counts()
contagem_target

segmento_de_cliente
Silver     260
Bronze     202
Starter     22
Gold        16
Name: count, dtype: int64

In [11]:
# Criar uma lista ordenada do target
lista_segmentos = ['Starter', 'Bronze', 'Silver', 'Gold']

In [12]:
# distribuição da variavel - Contagem
px.bar(contagem_target,color=contagem_target.index ,
       category_orders={'segmento_de_cliente': lista_segmentos}, 
       title='Distribuição da variável target')

In [13]:
# Distribuição da variavel Target - Percentual
percentual_target = contagem_target / len(df_segmento) * 100
px.bar(percentual_target,color=percentual_target.index ,
       category_orders={'segmento_de_cliente': lista_segmentos}, 
       title='Distribuição da variável target')

In [15]:
# Distribuição da variavel Localização
percentual_localizacao = df_segmento.value_counts('localizacao') / len(df_segmento) * 100
px.bar(percentual_localizacao,color=percentual_localizacao.index ,
       title='Distribuição da variável target')

In [16]:
# Distribuição da variavel Atividade economica
percentual_atividade = df_segmento.value_counts('atividade_economica') / len(df_segmento) * 100
px.bar(percentual_atividade,color=percentual_atividade.index ,
       title='Distribuição da variável target')

In [17]:
# Distribuição da variavel Inovação
percentual_inovacao = df_segmento.value_counts('inovacao') / len(df_segmento) * 100
px.bar(percentual_inovacao,color=percentual_inovacao.index ,
       title='Distribuição da variável target')

In [ ]:
# Tabela de contingência entre localização e Target (segmento de cliente)
crosstab_localizacao = pd.crosstab(df_segmento['localizacao'], 
                                   df_segmento['segmento_de_cliente'],
                                     margins=True)[lista_segmentos].reset_index()

tabela_localizacao = ff.create_table(crosstab_localizacao)
#mostrar a crosstab
tabela_localizacao.show()

In [24]:
# Tabela de contingência entre atividade economica e Target (segmento de cliente)
crosstab_atividade = pd.crosstab(df_segmento['atividade_economica'], 
                                 df_segmento['segmento_de_cliente'], 
                                 margins=True)[lista_segmentos].reset_index()

tabela_atividade = ff.create_table(crosstab_atividade)
#mostrar a crosstab
tabela_atividade.show()

In [25]:
# Tabela de contingência entre inovação e Target (segmento de cliente)
crosstab_inovacao = pd.crosstab(df_segmento['inovacao'], 
                                df_segmento['segmento_de_cliente'], 
                                margins=True)[lista_segmentos].reset_index()
tabela_atividade = ff.create_table(crosstab_inovacao)
#mostrar a crosstab
tabela_atividade.show()